# StrangleBot
We are building a trading bot to execute short strangles and long strangles on S&P 500 companies based on features we can extract from yfinance data.

## Setup

`git clone https://github.com/PeterFavero/cqf_final_project.git`

`cd cqf_final_project`

`python3 -m venv venv`

MacOS/Linux: `source venv/bin/activate` or Windows: `venv\Scripts\activate`

`pip3 install -r requirements.txt`

In [127]:
# IMPORTS
# ---------------------------------------------

import yfinance as yf
import pandas as pd
import pytz
import ssl
from datetime import datetime, timedelta
from tqdm import tqdm
import sys
import os
import numpy as np

In [128]:
# ENVIROMENT CONFIGURATION
# ---------------------------------------------
# This is nescessary for web scraping

ssl._create_default_https_context = ssl._create_unverified_context

In [129]:
# DATA: HELPER FUNCTIONS 
# ---------------------------------------------

def std_of_history(price_hist):
    price_hist['Returns'] = np.log(price_hist['Close'] / price_hist['Close'].shift(1))  # Calculate daily returns
    price_hist.drop(price_hist.index[0], inplace=True)
    volatility = price_hist['Returns'].std() * np.sqrt(252)  # Calculate the standard deviation of returns, annualizing the volatility
    return volatility

def data_of_tickers(full_ticker_strings_list) :
    cleaned_ticker_strings_list =  [ticker for ticker in full_ticker_strings_list if "." not in ticker]
    vol_dict = {}
    yf_dict = {}
    history_dict = {}
    for ticker in tqdm(cleaned_ticker_strings_list, desc="Downloading price history data") :
        yf_dict[ticker] = yf.Ticker(ticker)
        history_dict[ticker] = yf_dict[ticker].history(period='2y').drop('Dividends', axis=1).drop('Stock Splits', axis=1)
        annualized_volatility = std_of_history(history_dict[ticker])
        vol_dict[ticker] = annualized_volatility
    return cleaned_ticker_strings_list, vol_dict, yf_dict, history_dict 


In [130]:
# DATA: GET TICKERS LIST
# ---------------------------------------------

sp500_unprocessed = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].to_list()

In [131]:
# DATA: DATA LOADING
# ---------------------------------------------

tickers_list, vol_dict, yf_dict, history_dict = data_of_tickers(sp500_unprocessed)

In [132]:
history_dict['MMM']

,Open,High,Low,Close,Volume,Returns
Date,,,,,,
2022-05-05 00:00:00-04:00,112.654598,113.007631,109.021293,109.925941,3965816,-0.031416
2022-05-06 00:00:00-04:00,109.786190,110.153933,107.888634,109.977417,2954479,0.000468
2022-05-09 00:00:00-04:00,109.175731,113.095883,108.807987,112.073555,4979068,0.018880
2022-05-10 00:00:00-04:00,113.265044,113.294466,108.955089,109.675865,4189827,-0.021626
2022-05-11 00:00:00-04:00,110.315744,110.418713,107.506185,107.682701,3286608,-0.018340
...,...,...,...,...,...,...
2024-04-29 00:00:00-04:00,91.559998,92.620003,91.559998,92.160004,4905000,0.003587
2024-04-30 00:00:00-04:00,95.730003,97.839996,93.779999,96.510002,16731900,0.046120
2024-05-01 00:00:00-04:00,98.099998,99.699997,97.279999,98.440002,11062600,0.019801


In [133]:
print(vol_dict['MMM'])

0.28701395668747853
